# Effortless models deployment with Mlflow

**Author:** [Facundo Santiago (post link)](https://santiagof.medium.com/effortless-models-deployment-with-mlflow-customizing-inference-e880cd1c9bdd)<br>
**Editor:** [neu.ro](https://neu.ro)<br>
**Date created:** 2022/03/16<br>
**Last modified:** 2022/08/23<br>

**Description:** Training an image classifier from scratch on the Kaggle Cats vs Dogs dataset, storing model in MLFLow and deploying it to Neu.ro platform.

In the previous example, I showed how, by adopting MLFlow for logging models in your experiments, you can gain free deployment options in a variety of targets. However, one of the limitation presented with that is that you have to get along with the default way inference is proposed for the model flavor you are using.

For instance, if you are using fastai, then models will return the probabilities returned by the model directly. But what about if you have to do something different with those outputs? What about if you have to do some preprocessing before submitting the data to the model? In this example, I will show how can log the model using the `pyfunc` flavor to customize how inference is run. Particularly, we will change the output values for the model to return 2 columns: `class`, meaning the class index number (0 or 1), and `label`, meaning the label associated with the given class index.

### Solving the problem

Let's start by getting some undestanding of the problem. This is what we know:

<ul>
    <li><b>Problem type:</b> Classification</li>
    <li><b>Number of classes:</b> 2 (cats, dogs)</li>
    <li><b>Input:</b> Images (25.000 — 50% cats, 50% dogs)</li>
</ul>

The dataset is completely balanced. 

## Introducing fast.ai

We are introducing here a framework called fast.ai (https://www.fast.ai/), a framework based on PyTorch with some handy operations already implemented to speed up problem solving quickly. To use fast.ai, we need to import 2 libraries: fastai and torch. fast.ai also has the named dataset already uploaded as part of the framework, which makes pretty convenient to work with it.

> Since fast.ai is a pretty unstable API (it changes very frequently), we pin a specific version: `fastai==2.4.1`.

<div class="alert alert-block alert-warning"> <b>IMPORTANT</b> This notebook is designed to run on the Neu.ro platform, within the Jupyter server running as Application. MLFlow Application should be started beforehand and integrated with this Jupyter server App.

Let's install FastAI framework, since it is not included to the base image:

In [ ]:
%pip install fastai==2.4.1

Importing some libraries

In [ ]:
import torch
import fastai
import numpy as np
import pandas as pd

Since this is a computer vision problem, let's load some of the specific classes in FastAI for Computer Vision

In [ ]:
from fastai.vision.all import *
from fastai.vision.data import ImageDataLoaders

## Getting started

First things first, let's download and unzip the dataset. Please take into consideration that this dataset is a bit big. Notice path `/var/storage` is the place, where your user's storage home directory is mounted to persist changes.  

In [ ]:
path = untar_data(URLs.DOGS, dest="/var/storage/.fastai")
image_files = get_image_files(path)
print('Data located at:', path)

As images are unzipped in a folder, we need to create a dataset to use for training and testing. `fast.ai` has a very simple way to do that. The method `ImageDataLoader.from_folder` creates a dataset of images located in a folder. The folder itself has a pretty peculiar structure (imagenet format) where the data is already divided in trian/test and labels are inferred from the folder containing the image (folder cats contain `cats` and folder dogs contains `dogs`.

`batch_tfms` indicates which image transformations to apply in batches to all the images, while `item_tfms` indicate which transformations will be applied individually to each image. This is useful to construct transformation pipelines right into the data loader. The ds_tfms() method quickly gets a set of random transforms that have proved to work well in a wide range of tasks in computer vision, including a random flip is applied with probability 0.5, a random rotation, a random zoom, a random lightning and contrast change and a random symmetric warp.

Finally, the function normalize creates a normalize/denormalize func using an specific mean and std. In this case, those parameters are taken from the imagenet dataset, using the values imagenet_stats which are means = [0.485, 0.456, 0.406] and stds = [0.229, 0.224, 0.225].

In [ ]:
data_loader = ImageDataLoaders.from_folder(path, item_tfms=Resize(224), batch_tfms=Normalize.from_stats(*imagenet_stats))

In [ ]:
data_loader.train.show_batch()

### Creating a classifier based on ResNet34

#### Starting the mlflow experiment

Before starting, let's create a new experiemnt in Mlflow so we can track all the experimentation on it using Mlflow Tracking.

In [ ]:
import mlflow

experiment_id = mlflow.set_experiment("mlflow-fastai-cats-vs-dogs")

Let's start the run

In [ ]:
mlflow.start_run()

> **Note:** In general, we would run the training code in a `with` block using `with mlflow.start_run()`. This will ensure that everything in the with block is part of the run and hence mlflow will be able to track the time it takes to run the experiment. Since we are running this interactively in a notebook, we have to run the `start_run()` method at the beginning and `end_run()` at the end.

Turn on `Autolog()`. This will enable MLFlow go gather all training metrics automatically (checkout [MLFlow docs](https://www.mlflow.org/docs/latest/python_api/mlflow.fastai.html#mlflow.fastai.autolog) for more information). 

In [ ]:
mlflow.fastai.autolog(log_models=False)

> **Why `log_models` is set to `false`?** In this example, we are going to change the way MLFlow runs inference for the model. If we tell MLFlow to automatically log the model, then we won't have a chance to change the default behavior. By disabling auto log of models, metrics and parameters will continue to be automatically tracked but models needs to be log manually.

#### Designing a CNN
Once our dataset is ready, it's time to create our neural network. CNN represents a very convenient way to solve Computer Vision problems, specially when combined with transfer learning. We use transfer learning with a pretrained image classification models to extract visual features. The idea behind it is that the representations learned for task A (typically a high-level task) are applied to task B (typically a lower-level task) as for the degree of success at task B indicates how much the task A model has learned about task B.

In [ ]:
learn = cnn_learner(data_loader, models.resnet34, metrics=accuracy)

Then it's time to train. When using transfer learning, the training process is a bit different like in a normal network. In the processes we take a pre-trained model and “fine-tuning” the model with your our own dataset. The idea is that this pre-trained model will act as a feature extractor. You will remove the last layer of the network and replace it with your own classifier. You then freeze the weights of all the other layers and train the network normally

In [ ]:
suggested_lr = learn.lr_find()

I'm using here the method `learn.lr_find()`. What this method does behind the scenes is running for few epochs to find out a good learning rate, where it trains from some low learning rate and increase the learning rate after each mini-batch till the loss value starts to explode. This single run provides valuable information on how well the network can be trained over a range of learning rates and what is the maximum learning rate. This is based on a paper https://arxiv.org/abs/1506.01186 which is a really good reading by the way. In Cyclical learning rates (CLR) one specifies minimum and maximum learning rate boundaries and a stepsize. The stepsize is the number of iterations (or epochs) used for each step and a cycle consists of two such steps – one in which the learning rate linearly increases from the minimum to the maximum and the other in which it linearly decreases.

We can then see which is the suggested learning rate by using the property `valley`. Let's use that to train our model.

In [ ]:
learn.fine_tune(1, suggested_lr.valley)

> For simplicity, I'm only training for 1 epoch

### Testing the trained model

Let's see if the model works with an image

In [ ]:
from fastai.vision.core import load_image

sample_img = load_image(image_files[1])
sample_img_arr = np.array(sample_img)

> Resizing the image is not required, as the model can do that in its trainsformation's pipeline. The reason why I do this is a bit bellow.

Running the model:

In [ ]:
dl_model = learn.dls.test_dl([sample_img_arr])
real_preds, _ = learn.get_preds(dl=dl_model)

In [ ]:
print(real_preds)

To obtain the classes rather than the probabilities, we can do something like this:

In [ ]:
class_idxs = real_preds.argmax(axis=1)
print(class_idxs)

In [ ]:
res = [(c.numpy(), learn.dls.vocab[c]) for c in class_idxs]
print(res)

### Persisting the model

Once the model is trained, it's time to save the work. 

> **Note:** An important distinction here, `learn.save` will save the model itself in the `PyTorch` format, while the method `learn.export` will save both the model along with all the data transformations applied. This is important since at inference time our model expects to recive images that has been altered the same way the previous data loaded did.

In [ ]:
saved_model = "/tmp/model.fastai"
learn.export(saved_model)

#### Creating the signature

The model will take batches of images and will return the class of each (in our case either cat or dog). This means that our inputs will be tensors of shape `(IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS)`. Since the model expects batches of images, the right input shape is `(BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS)`.

For our output, we want to return both the class index and the class label. Then, our output signature will have a shapre `(-1, 2)` but with an specific struncture: two columns: `class` and `label`.

The schema then will be as follows:

In [ ]:
import numpy as np
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec, ColSpec
from mlflow.types import DataType

input_schema = Schema([
  TensorSpec(np.dtype(np.uint8), (-1, -1, -1, 3)),
])
output_schema = Schema([
  ColSpec(DataType.integer, "class"),
  ColSpec(DataType.string, "label"),
])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

Some notes:

- **input_schema**: Our model can handle variable image sizes, since it has a transformation that resizes the image in the process (the `Resize(224)` transformation). Then, we can indicate a variable input size using -1 in `height` and `width`. The same applies to batch size, since we can handle variable sizes.

- **output_schema**: Our model will return two columns. We are not using tensor specification cause we want to be explicit about what are the things we are returning, so we can see we are actually indicating which is the name of the columns we return.

#### Saving the model in the Mlflow format

To save the model using Mlflow, we can call the function `mlflow.fastai.log_model`. However, this function will run the model and return its output directly, without any further processing. We can change this behaviour by using the `pyfunc` flavor:

We can use the pyfunc flavor in two ways:

- The simplest way is to provide a custom model loader module.
- Create a custom flavor. This one is a bit more complex.

**In this example we will use the first one**. To use `pyfunc` flavor with a custom model loader, we are asked to implement two things:

- a class that acts as a model wrapper. This class has to have a method `predict()`.
- a model loader function called `_load_pyfunc(path)` where path is the path to the models' artifact and that returns an instance of the previously mentioned class.

Let's have a lookg at how we can achieve this:

First let's create the model class:

In [ ]:
class _FastaiClassifierModelWrapper:
    def __init__(self, learner):
        self.learner = learner

    def predict(self, data):
        import pandas as pd
        import numpy as np
        
        dl = self.learner.dls.test_dl(data)
        preds, _ = self.learner.get_preds(dl=dl)
        
        # converting probabilities to classes
        class_idxs = preds.argmax(axis=1)
        results = [(c.numpy(), self.learner.dls.vocab[c]) for c in class_idxs]
        
        return pd.DataFrame(results, columns=["class", "label"])

This simple class contains:

- A constructor that recieves an instance of a `fastai.learner` model.
- The predict function we are required to implement. This function recieves a data an returns the predictions in a dataframe with columns named `class` and `label`. 

Finally, we need to provide a way to load this model. Again, this is done by implementing the function `_load_pyfunc()`:

In [ ]:
def _load_pyfunc(path):
    import os
    from fastai.learner import load_learner
    
    learn = load_learner(os.path.abspath(path))
    
    return _FastaiClassifierModelWrapper(learn)

That's all! Now let's place this inside of a module so we can use it later:

In [ ]:
%%writefile fastai_classifier_module.py

class _FastaiClassifierModelWrapper:
    def __init__(self, learner):
        self.learner = learner

    def predict(self, data):
        import pandas as pd
        import numpy as np
        
        dl = self.learner.dls.test_dl(data)
        preds, _ = self.learner.get_preds(dl=dl)
        
        # converting probabilities to classes
        class_idxs = preds.argmax(axis=1)
        results = [(c.numpy(), self.learner.dls.vocab[c]) for c in class_idxs]
        
        return pd.DataFrame(results, columns=["class", "label"])

def _load_pyfunc(path):
    import os
    from fastai.learner import load_learner
    
    learn = load_learner(os.path.abspath(path))
    
    return _FastaiClassifierModelWrapper(learn)

Now it's time to log the model in MLFlow. We also include `fastai` version as extra pip requirements.

In [ ]:
model_info = mlflow.pyfunc.log_model(
    "classifier", 
    data_path=saved_model, 
    code_path=["./fastai_classifier_module.py"], 
    loader_module="fastai_classifier_module", 
    registered_model_name="cats_vs_dogs", 
    signature=signature,
    extra_pip_requirements=["fastai==2.4.1"]
)

Let's unpack how this instruction works:

- **classifier** is the artifacts path we will use. Same as before, we are using the path `classifier`.
- **data_path** is the model artifact. When we used `mlflow.fastai.log_model`, MLFlow automatically saved our model to disk to then include it inside the MLModel package. However, when we use `pyfunc`, this doesn't happen automatically. We have full controll of it so we have to persist the model to disk. We did that before, and we saved the path where the model was persisted in a variable called `saved_model`. 
- **code_path** is where the code for the `pyfunc` logic is located. We placed all the logic inside of the file `fastai_classifier_module.py`. So we pass that file path.
- **loader_module** is the name of the module where the `_load_pyfunc` function is defined. In our case, that function is inside the file `fastai_classifier_module.py`. Remember that in Python, files are modules and the name of the associated module is the name of the file, so the name of the module here is `fastai_classifier_module`. If you are thinking "this looks redundant", it worth to mention that code_path can be the path to a directory, containing multiple code files. MLFLow will snapshot all the files in that directory, so it needs to know which of all those files (in our case is just one) is the one it has to load first.
- **registered_model_name** is the name of a model in MLFlow model registry, to which this particular model version should be assigned.
- **signature** is the description of model inputs and outputs used during the inference.
- **extra_pip_requirements** is a list of Python requirements needed to run the model. Usually, MLFlow infers this automatically, but when it fails (or you have some extra requirements), this becomes handy, for instance, to pin some package to a specific version.

### Ending the experiment

Since we were running this experiment interactively in this notebook, we have to end it using:

In [ ]:
mlflow.end_run()

## Deploying your model using mlflow

Now, let's see how we can deploy the model using Mlflow. Before doing that, it is always good to check that our model was persisted correctly and that it can be loaded. This will save us some time later in case something wrong happens.

Let's grab the URI of the model we just registered. 

In [ ]:
serving_model_uri = model_info.model_uri
print(serving_model_uri)

That's the URI of the model inside the run (which we can totally deploy if we want to). However, I'm interested in deployed the model that ended registered in the Registry. This model will have a URI with the form `models:/cats_vs_dogs/latest`.

In [ ]:
serving_model_uri = 'models:/cats_vs_dogs/latest'

> `latest` will resolve the latest version of the given model. If you want an specific version of the model, we can use a syntax like `models:/cats_vs_dogs/1` for version `1` for instance.

### Loading the model just logged in MLFLow using the pyfunc flavor.

First, let's try to load the model using the `pyfunc` flavor into this notebook. We can use this method to check that the model can be loaded correctly.

In [ ]:
mlflow_model = mlflow.pyfunc.load_model(serving_model_uri)

Let's create a simple batch to test it out:

In [ ]:
sample_batch = sample_img_arr.reshape(1, sample_img.height, sample_img.width,3)

In [ ]:
mlflow_model.predict(sample_batch)

> Note how we use here the method `predict` to run the predictions, but now we get classes instead of probabilities. Compare this with the way we used to run the model before in the section [Testing the trained model](#Testing-the-trained-model)

### Running the model in a local server

We can run the model in an inference server locally in our local compute. Again, with this we can check that our deployment strategy will work. 

To do so, execute this in a dedicated Jupyter server console:

```bash
mlflow models serve -m models:/cats_vs_dogs/latest
```

Creating a sample request

In [ ]:
import json

with open("sample.json", "w") as f:
    f.write(json.dumps(
        {
            "instances": sample_batch.tolist()
        }
    ))

Sending the request

In [ ]:
!cat -A sample.json | curl http://127.0.0.1:5000/invocations \
                        --request POST \
                        --header 'Content-Type: application/json' \
                        --data-binary @-

## Deploying the model to Neu.ro platform

MLflow is integrated with several Machine Learning platforms, including Neu.ro MLOps platform. We can deploy models created with MLFlow without changing any single implementation about them. They can be deployed using [Job Deployment Application](https://github.com/neuro-inc/mlops-job-deploy-app).

Open platform Web inferface, navigate to Applications Dashboard and start Job Deployment Application attached to the MLFlow instance you used with this Jupyter instance.

When the Application is started, create new deployment for previously registered model version using MLFlow as a target server, and disable authentication (for tests purposes). Use `mlflow` or `base` images to serve the model.

When the model server completes environment creation and model startup, use it's Endpoint URL to query the model with the same method:

In [ ]:
!cat -A sample.json | curl <endpoint-url>/invocations \
                        --request POST \
                        --header 'Content-Type: application/json' \
                        --data-binary @-

## Cleanup
When you are done with the tutorial, terminate everything you created for it: deployed inference server, Job Deployment Application, Jupyter, and MLFlow servers.